# Aufgabe: Ist eine Iris mit den Werten [ 4.8,2.5,5.3,2.4 ] eine Iris-Virginica? (Mit KNN)
Zuerst: Umgebung einrichten und einen Überblick über das Dataset erhalten:

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets, neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
iris = datasets.load_iris()
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

----------
Keys und features herausfinden:

In [2]:
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [3]:
iris["feature_names"]

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

--------
Wir wandeln den Datensatz in einen pandas data frame um, damit wir ihn besser handlen können und gucken uns den Anfang des data frames an:

In [4]:
iris_df = pd.DataFrame(iris.data)
iris_df.columns = iris.feature_names
iris_df['classes'] = iris.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),classes
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


--------
Wir weisen unserem zu klassifizierenden Datensatz eine Variable zu:

In [5]:
to_classify = np.array([4.8, 2.5, 5.3, 2.4]).reshape(1, -1)
to_classify

array([[4.8, 2.5, 5.3, 2.4]])

--------
Wir instanziieren einen KNN classifier mit k=10, mit dem wir die 10 nächsten Nachbarn bestimmen können:

In [6]:
classifier_1 = neighbors.NearestNeighbors(10)
classifier_1.fit(iris_df.drop(['classes'], 1))

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

--------
Wir geben unseren Datensatz in den Klassifier und lassen uns das Ergebnis anzeigen (Die Ergebnisse sind bereits nach Abstand aufsteigend sortiert):

In [7]:
result = classifier_1.kneighbors(to_classify, 10)
abstand = result[0][0]
nn = result[1][0]
print('Abstand:\n', abstand, '\n\nDatensatz-Index:\n', nn)

Abstand:
 [1.02469508 1.02956301 1.06301458 1.06770783 1.15325626 1.15325626
 1.36381817 1.43527001 1.46969385 1.51657509] 

Datensatz-Index:
 [121 113 114 106 101 142 149  84  83 138]


--------
Wir lassen uns nun die Datensätze mit den entsprechenden Indizes anzeigen:

In [8]:
iris_df.loc[nn]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),classes
121,5.6,2.8,4.9,2.0,2
113,5.7,2.5,5.0,2.0,2
114,5.8,2.8,5.1,2.4,2
106,4.9,2.5,4.5,1.7,2
101,5.8,2.7,5.1,1.9,2
142,5.8,2.7,5.1,1.9,2
149,5.9,3.0,5.1,1.8,2
84,5.4,3.0,4.5,1.5,1
83,6.0,2.7,5.1,1.6,1
138,6.0,3.0,4.8,1.8,2


--------
Wir können leicht erkennen, dass die 7 Datensätze mit dem kleinsten Abstand alle zur Klasse 2 gehören. Klasse 2 ist Virginica, wie wir ganz oben in diesem Notebook sehen können:
 - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
Die Datensätze 84 und 83 gehören zu Klasse 1, der letzte Datensatz 138 scheint ein Outlier in Klasse 2 zu sein. Wir können also mit großer Wahrscheinlichkeit bestimmen, dass unser Testdatensatz zur Klasse 2 gehört und damit eine Virginica ist.

--------
# Alternativer Ansatz:
Zuerst zerlegen wir den Datensatz in Trainings (80 %)- und Testdaten (20 %) und implementieren KNN. Es werden jedesmal andere Datensätze zufällig ausgewählt. Wir berechnen außerdem die Wahrscheinlichkeit, das die Vorhersage, die auf der Basis der soeben erstellten Trainings- und Testdaten erfolgt, richtig ist:

In [9]:
x_train, x_test, y_train, y_test = train_test_split(
                    np.array(iris['data']),
                    np.array(iris['target']),
                    test_size=0.2)

classifier_2 = KNeighborsClassifier(n_neighbors=7) 
classifier_2.fit(x_train, y_train) 
prediction_train = classifier_2.predict(x_test)
print("Testdaten:  ", y_test)
print("Vorhersage: ", prediction_train)

def calculate_accuracy(prediction_train, target):
    counter = 0
    for i in range(len(prediction_train)):
        if prediction_train[i] == target[i]:
            counter += 1
    return counter / len(prediction_train)
print("Die Wahrscheinlichkeit beträgt: ", calculate_accuracy(prediction_train, y_test))

Testdaten:   [0 0 2 2 2 0 0 1 1 0 1 0 0 1 2 2 1 2 0 0 2 2 2 1 2 2 1 1 0 0]
Vorhersage:  [0 0 2 2 2 0 0 1 1 0 1 0 0 1 2 1 1 2 0 0 2 2 2 1 1 2 1 1 0 0]
Die Wahrscheinlichkeit beträgt:  0.9333333333333333


------
Nun geben wir unseren Testdatensatz in den classifier:

In [10]:
prediction = classifier_2.predict(to_classify)
print("Die Klasse ist:", prediction[0], "(", iris.target_names[prediction[0]], ")", ", mit der Wahrscheinlichkeit:", calculate_accuracy(prediction_train, y_test))


Die Klasse ist: 2 ( virginica ) , mit der Wahrscheinlichkeit: 0.9333333333333333


# Die Antwort lautet: Klasse 2 und damit Virginica.